# <font color=#0404B4>**SEMINAR - PHÂN TÍCH DỮ LIỆU THÔNG MINH** </font>

### <font color=#0404B4> Bảng thành viên nhóm </font>
|<center><div style="width:150px">MSSV</div><center>|<center><div style="width:290px">Họ và tên</div><center>|
|---------- |:-------------:|
| <center>20120032<center>  | <center>Phan Trường An<center> |
| <center>20120037<center>  | <center>Trần Thị Minh Anh<center> |
| <center>20120041<center>  | <center>Trần Kim Bảo<center> |
| <center>20120053<center>  |   <center>Nguyễn Thành Đạt<center>   | 
| <center>20120071<center>  |<center>Nguyễn Thị Bích Hà<center> |
| <center>20120095<center>  | <center>Ngô Quang Hưng<center> |

# <font color=#0404B4>Import thư viện</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
sns.set_theme()
pd.set_option("display.max_colwidth", 250)

In [2]:
!pip install locationtagger
!pip install geotext
!pip install geopy

In [3]:
from geotext import GeoText
from geopy.geocoders import Nominatim

<a name="2"></a>
# <font color=#0404B4> A. Tiền xử lý dữ liệu</font>


### 1. Đọc dữ liệu

In [4]:
train_df=pd.read_csv("datasets/train.csv")
train_df.shape

(7613, 5)

In [5]:
test_df=pd.read_csv("datasets/test.csv")
test_df.shape

(3263, 4)

In [6]:
sample_submission_df=pd.read_csv("datasets/sample_submission.csv")
sample_submission_df.shape

(3263, 2)

In [7]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
sample_submission_df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


### 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- `train_df`: Thông tin của tweet trên tập huấn luyện gồm: id, từ khóa, địa điểm gửi, nội dung và nhãn của tweet.
- `test_df`: Thông tin của tweet trên tập test gồm: id, từ khóa, địa điểm gửi, nội dung.
- `sample_submission_df`: định dạng nộp bài.

### 3. Có dữ liệu bị trùng lặp ở các dataframe không

In [10]:
def check_duplicated(dataframe):
    duplicated_rows = dataframe.duplicated()

    # In các hàng bị lặp lại
    if len(dataframe[duplicated_rows])>0:
        print('Có dòng dữ liệu bị lặp')
        display(dataframe[duplicated_rows])
    else: 
        print('Không có dòng dữ liệu nào bị lặp')


Dataframe `train_df`

In [11]:
check_duplicated(train_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `test_df`

In [12]:
check_duplicated(test_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `sample_submission_df`

In [13]:
check_duplicated(sample_submission_df)

Không có dòng dữ liệu nào bị lặp


<h3> 4. Mỗi cột có ý nghĩa gì?</h3>

|STT|Tên cột dữ liệu | <center>Mô tả<center> | Ghi chú |
|--|:------:| ---| --- |
|1|id|<center>định danh của tweet<center>|
|2|keyword| <center>Từ khóa được lấy từ tweet<center>|Có thể thiếu|
|3|location|<center>Địa điểm đăng tweet<center>| Có thể thiếu|
|4|text| <center>Nội dung của tweet<center>||
|5|target|<center>Nhãn được gán cho tweet<center>|1: Thảm họa thực sự <br> 0: Không xảy ra thảm họa| 



### 5. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp?



Dataframe `train_df`

In [14]:
train_df.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Dataframe `test_df`

In [16]:
test_df.columns

Index(['id', 'keyword', 'location', 'text'], dtype='object')

In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


- Nhận xét:
    - Dựa vào ý nghĩa của id và kích thước của `train_df` và `test_df`, nhận thấy rằng chúng ta có thể merge dữ liệu ở 2 dataframe để tạo thành 1 dataframe mới: `tweet_df`. Để tiện cho phần khám phá dữ liệu.

In [18]:
tweet_df=pd.concat([train_df,test_df])

In [19]:
tweet_df=tweet_df.sort_values(by="id").reset_index(drop=True)

In [20]:
check_duplicated(tweet_df)

Không có dòng dữ liệu nào bị lặp


In [21]:
tweet_df.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [22]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10876 entries, 0 to 10875
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        10876 non-null  int64  
 1   keyword   10789 non-null  object 
 2   location  7238 non-null   object 
 3   text      10876 non-null  object 
 4   target    7613 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 425.0+ KB


### 6. Với mỗi cột, các giá trị được phân bố như thế nào?

#### Cột có kiểu dữ liệu dạng số, và phân bố:


In [23]:
numerical_cols = tweet_df[tweet_df.select_dtypes(exclude=['object']).columns]
numerical_cols.head(5)

,id,target
0,0,NaN
1,1,1.0
2,2,NaN
3,3,NaN
4,4,1.0


- Với các cột dữ liệu dạng số ta sẽ tính % dữ liệu thiếu, min, max. Kết quả được lưu vào dataframe `nume_col_profiles_df`, với:
    - `missing_ratio`: % dữ liệu thiếu.
    - `min`: giá trị nhỏ nhất.
    - `max`: giá trị lớn nhất.

In [24]:
numerical_cols = tweet_df[tweet_df.select_dtypes(exclude=['object']).columns]

def missing_ratio(c):
    return  c.isna().sum()/len(c) * 100

def min(c):
    return c.min()

def max(c):
    return c.max()

nume_col_profiles_df = numerical_cols.agg([missing_ratio, min, max])
nume_col_profiles_df

,id,target
missing_ratio,0.0,30.001839
min,0.0,0.000000
max,10875.0,1.000000


- Nhận xét:
    - `id`: Không có dữ liệu thiếu, ID tweet nhỏ nhất là 0, lớn nhất là 10875.
    - `target`: Có 30% dữ liệu thiếu tương ứng với các tweet chưa được gán nhãn ở `test_df`.

#### Cột có kiểu dữ liệu phân loại, và phân bố:


In [25]:
object_cols = tweet_df[tweet_df.select_dtypes(include=['object']).columns]
object_cols.head(5)

,keyword,location,text
0,NaN,NaN,Just happened a terrible car crash
1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
4,NaN,NaN,Forest fire near La Ronge Sask. Canada


- Với các cột categorical, ta tính tỉ lệ % dữ liệu thiếu, số lượng giá trị khác nhau, phân bố của các giá trị khác nhau. Kết quả được lưu vào dataframe `cate_col_profiles_df`, với:
    - `missing_ratio`: % dữ liệu thiếu
    - `num_diff_vals`: số lượng giá trị khác nhau <em>(không xét giá trị thiếu)</em>
    - `ratio_diff_vals`: phân bố của các giá trị khác nhau <em>(không xét giá trị thiếu)</em>.

In [26]:
def num_diff_vals(col):
    return len(col.value_counts())

def ratio_diff_vals(col):
    return np.round(col.value_counts(normalize=True),2).to_dict()

cate_col_profiles_df=object_cols.agg([missing_ratio,num_diff_vals,ratio_diff_vals])
cate_col_profiles_df

,keyword,location,text
missing_ratio,0.799926,33.449798,0.0
num_diff_vals,221,4521,10678
ratio_diff_vals,"{'ablaze': 0.0, 'lava': 0.0, 'loud%20bang': 0.0, 'mass%20murder': 0.0, 'massacre': 0.0, 'mayhem': 0.0, 'meltdown': 0.0, 'military': 0.0, 'mudslide': 0.0, 'natural%20disaster': 0.0, 'nuclear%20disaster': 0.0, 'nuclear%20reactor': 0.0, 'obliterate'...","{'USA': 0.02, 'New York': 0.02, 'United States': 0.01, 'London': 0.01, 'Canada': 0.01, 'Nigeria': 0.01, 'India': 0.0, 'Worldwide': 0.0, 'Los Angeles, CA': 0.0, 'UK': 0.0, 'Kenya': 0.0, 'Washington, DC': 0.0, 'Mumbai': 0.0, 'United Kingdom': 0.0, ...","{'11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...': 0.0, '#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestna..."


- Nhận xét:
    - `location`: xuất hiện nhiều tweet không có địa chỉ đăng(33,45%), số lượng địa điểm lớn: 4521 địa điểm.
    - `keyword`: vẫn xuất hiện các tweet không trích dẫn được từ khóa, số lượng không quá lớn: 221 cụm từ khóa.
    - `text`: Không bị thiếu dữ liệu, nhưng số lượng nội dung chỉ là 10678 < 10875 (số lượng bài tweet). Có thể là do 1 số sự kiện/vấn đề nào đó.

###  7. Có cột nào cần phải tiền xử lý hay không?


- Nhận xét:
    - Qua kiểm tra dữ liệu, ta có thể thấy ở thuộc tính `location` có rất nhiều giá trị(4521 địa điểm) xấp xỉ bằng 1/2 tổng 2 tập `train_df` và `test_df` cộng lại. Thực hiện kiểm tra và xử lý về dạng chuẩn nhất có thể.

#### 1. Xóa các địa điểm bị lỗi ký tự, xuất hiện ký tự đặc biệt nhiều:


Thực hiện xóa các hàm có ký tự đặc biệt nhiều hơn 1/4 so với độ dài của chuỗi. Và nếu trong cell đang xét có từ nào trên 2 ký tự thì thực hiện trả về *(Chỉ lấy 1 từ cuối)*

In [27]:
def del_special_string(c):
  if not(c is np.nan):
    special_char=re.sub('[\w^\s\.]+' ,'', c)
    scale=len(special_char)/len(c)
    if scale > 1/4:
      words=re.findall('[a-zA-Z]{2,}',c)
      if len(words)>0:
        return words[-1]
      return np.nan
    else:
      return c
  return np.nan

Thực hiện chạy hàm `del_special_string` cho tất cả các dữ liệu trong cột `location`

In [28]:
tweet_df.location.apply(del_special_string).str.strip().replace('',np.nan).unique().__len__()

4352

In [29]:
tweet_df.location=tweet_df.location.apply(del_special_string).str.strip().replace('',np.nan)

Đếm số lần địa điểm được sử dụng

In [30]:
location_count_df=tweet_df.location.value_counts().to_frame().reset_index().rename(columns={"location":"counts","index":"location"})
location_count_df

,location,counts
0,USA,146
1,New York,112
2,United States,65
3,London,60
4,Nigeria,46
...,...,...
4346,buenos aires argentina,1
4347,everydaynigerian@gmail.com,1
4348,"Surulere Lagos,Home Of Swagg",1
4349,"MontrÌ©al, QuÌ©bec",1


In [31]:
location_array=location_count_df.location

#### 2.  Lọc các dữ liệu có tên thành phố hoặc tên quốc gia

##### Trích những tên thành phố và quốc gia *(đúng chuẩn)*

Tạo dataframe chứa thông tin: thành phố và quốc gia dựa trên các địa chỉ *(unique)*

In [32]:
location_dict={"city":[],"country":[]}
for location_str in location_array:

  if isinstance(location_str, str):
    places = GeoText(location_str)

    if len(places.cities)>0:
      if len(places.cities)>1:
        result=', '.join(places.cities)
        location_dict["city"].append(result)
      else :
        location_dict["city"].append(places.cities[0])
    else :
        location_dict["city"].append(np.nan)

    if len(places.countries)>0:
      location_dict["country"].append(places.countries[0])
    else :
      location_dict["country"].append(np.nan)
  else:
    location_dict["city"].append(np.nan)
    location_dict["country"].append(np.nan)
  

In [33]:
location_df=pd.DataFrame(location_dict)
location_df

,city,country
0,NaN,NaN
1,New York,NaN
2,NaN,United States
3,London,NaN
4,NaN,Nigeria
...,...,...
4346,NaN,NaN
4347,NaN,NaN
4348,NaN,NaN
4349,NaN,NaN


##### Điền tên quốc gia cho các tweet chỉ có tên thành phố

In [34]:
def get_country(city):
  if not (city is np.nan):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(city)
    if location:
      return location.raw['display_name'].split(",")[-1].strip()
    return np.nan
  return np.nan

In [35]:
location_fill_country=location_df[~(location_df.city.isna()) & (location_df.country.isna())]

In [36]:
# Đã chạy và lưu kết quả vào file fill_country.csv
countries = location_fill_country.city.iloc[:5].apply(get_country)

location_df.loc[countries.index, 'country'] = countries.values
location_df

,city,country
0,NaN,NaN
1,New York,United States
2,NaN,United States
3,London,United Kingdom
4,NaN,Nigeria
...,...,...
4346,NaN,NaN
4347,NaN,NaN
4348,NaN,NaN
4349,NaN,NaN


Xuất file `fill_country.csv` sau khi đã điền: để thuận tiện cho các lần chạy sau

In [37]:
location_df["location"]=pd.Series(location_array)

In [38]:
# location_df.to_csv("fill_country.csv",index=False,header=True)

Đọc lại dữ liệu từ `fill_country.csv`

In [39]:
location_df=pd.read_csv("datasets/fill_country.csv")
location_df

,city,country,location
0,NaN,NaN,USA
1,New York,United States,New York
2,NaN,United States,United States
3,London,United Kingdom,London
4,NaN,Nigeria,Nigeria
...,...,...,...
4346,NaN,NaN,buenos aires argentina
4347,NaN,NaN,everydaynigerian@gmail.com
4348,NaN,NaN,"Surulere Lagos,Home Of Swagg"
4349,NaN,NaN,"MontrÌ©al, QuÌ©bec"


#### 3. Chuẩn hóa các thông tin bị trống cả tên thành phố và quốc gia

Lọc các tweet bị trống thông tin ở 2 cột thành phố và quốc gia

In [40]:
location_nan_all_df=location_df[(location_df.city.isna()) & (location_df.country.isna())]
location_nan_all_df

,city,country,location
0,NaN,NaN,USA
7,NaN,NaN,Worldwide
9,NaN,NaN,UK
12,NaN,NaN,California
20,NaN,NaN,"California, USA"
...,...,...,...
4344,NaN,NaN,todaysbigstock.com
4346,NaN,NaN,buenos aires argentina
4347,NaN,NaN,everydaynigerian@gmail.com
4348,NaN,NaN,"Surulere Lagos,Home Of Swagg"


Hàm trả về tên quốc gia cho location chưa chuẩn hóa *(nếu có đề cập)*

In [41]:
def get_country_nan_all(c):
  geolocator = Nominatim(user_agent='myencoder', timeout=10)
  first = geolocator.geocode(c)
  if first:
    place=GeoText(first.address)
    if len(place.countries)<1:
      return first.raw['display_name'].split(",")[-1].strip()
    else :
      return place.countries[0]
  else :
    return np.nan

In [42]:
location_count_df[location_count_df.counts>1].counts.sum()

3514

- Nhận xét:
    - Thực hiện chuẩn hóa cho các địa điểm xuất hiện nhiều hơn 1 lần.
    - Theo tìm hiểu để có thể lấy được tên các quốc gia chưa được chuẩn hóa trong ngôn ngữ tự nhiên. Nhóm quyết định xử dụng thư viện `geopy`: truy cập lấy từ API từ [OpenStreetMap Nominatim](https://nominatim.openstreermap.org). Cho phép lấy được cả các thông tin chưa được chuẩn hóa dựa trên gợi ý từ API. Tuy nhiên với điều khoản chỉ có thểm lấy 1 địa chỉ/s và bị giới hạn số lần yêu cầu. Nên khó khăn trong việc chuẩn hóa toàn bộ.


In [43]:
len_NanCountrySuggest=location_count_df[location_count_df.counts>1].shape[0]

In [44]:
location_nan_all_df[location_nan_all_df.index<len_NanCountrySuggest]

,city,country,location
0,NaN,NaN,USA
7,NaN,NaN,Worldwide
9,NaN,NaN,UK
12,NaN,NaN,California
20,NaN,NaN,"California, USA"
...,...,...,...
719,NaN,NaN,far away
723,NaN,NaN,hyderabad
725,NaN,NaN,434
726,NaN,NaN,"sydney, australia"


In [45]:
start_idx=0
end_idx=100
steps=100
# n=location_nan_all[location_nan_all.index<len_NanCountrySuggest].shape[0]
n=0
while(True):
    if end_idx>n:
        countries = location_nan_all_df.location.iloc[start_idx:n].apply(get_country_nan_all)
        location_df.loc[countries.index, 'country'] = countries.values
        break
    countries = location_nan_all_df.location.iloc[start_idx:end_idx].apply(get_country_nan_all)
    location_df.loc[countries.index, 'country'] = countries.values
    print(end_idx)
    start_idx+=steps
    end_idx+=steps

Xuất file `fill_locatioin.csv` sau khi đã điền: để thuận tiện cho các lần chạy sau

In [46]:
# location_df.to_csv("fill_location.csv",index=False,header=True)

In [47]:
location_nomalize_df=pd.read_csv("datasets/fill_location.csv")

In [48]:
location_nomalize_df.head(10)

,city,country,location
0,NaN,United States,USA
1,New York,United States,New York
2,NaN,United States,United States
3,London,United Kingdom,London
4,NaN,Nigeria,Nigeria
5,NaN,Canada,Canada
6,NaN,India,India
7,NaN,United Kingdom,Worldwide
8,Los Angeles,United States,"Los Angeles, CA"
9,NaN,United Kingdom,UK


In [49]:
location_nomalize_df.loc[location_nomalize_df.location=="Worldwide","country"]="Worldwide"

Thực hiện chuẩn hóa dữ liệu vào tập dữ liệu `train_df` và `test_df`

In [50]:
train_df = train_df.merge(location_nomalize_df, how='left',on ="location")
test_df = test_df.merge(location_nomalize_df, how='left',on ="location")

In [54]:
train_df=train_df[["id", "keyword", "city", "country", "location", "text", "target"]]

In [55]:
train_df.head()

,id,keyword,city,country,location,text,target
0,1,NaN,NaN,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [56]:
test_df=test_df[["id", "keyword", "city", "country", "location", "text"]]

In [57]:
test_df.head()

,id,keyword,city,country,location,text
0,0,NaN,NaN,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


<a name="2"></a>
# <font color=#0404B4> B. Khám phá dữ liệu</font>
